### Loading trained model (MNIST)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

***

#### Network definition

| Layer     | Output Tensor | Weight          | Bias | Note    |
|:---------:|--------------:|----------------:|-----:|---------|
| input     | 28 x 28 x 1   | -               | -    |         |
| conv 1    | 24 x 24 x 10  | 5 x 5 x 1 x 10  | 10   | w/ ReLU |
| maxpool 1 | 12 x 12 x 10  | -               | -    |         |
| conv 2    | 8 x 8 x 20    | 5 x 5 x 10 x 20 | 20   | w/ ReLU |
| maxpool 2 | 4 x 4 x 20    | -               | -    |         |
| dropout   | 4 x 4 x 20    | -               | -    |         |
| fc 1      | 320 x 1       | 320 x 50        | 50   |         |
| fc 2      | 50 x 1        | 50 x 10         | 10   |         |
| output    | 10            | -               | -    |         |

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

#### Setting

In [3]:
data_dir = 'data'
model_path = 'mnist.pt'
test_batch_size = 10

#### Use GPU?

In [4]:
use_cuda = torch.cuda.is_available()
print("Using GPU..." if use_cuda else "NOT using GPU...")

device = torch.device("cuda" if use_cuda else "cpu")

Using GPU...


***

#### Load model

In [5]:
param = torch.load(model_path)
model = Net().to(device)
model.load_state_dict(param)

<All keys matched successfully>

***

#### Inference

In [6]:
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(data_dir, train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=test_batch_size, shuffle=True, num_workers=2)

In [7]:
# for ite in range(1, 10000/test_batch_size):
test_loss = 0
model.eval()
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

D:\Program\miniconda3\envs\pytorch\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [8]:
print("Accuarcy = %.3f [%%]" % ( float(correct)/10000*100)  )

Accuarcy = 98.660 [%]
